### Current Progress/Ideas:

Using deepface to analyze images is too slow, e.g. takes 20-30 seconds per image. Likely unable to use for short demo unless significant efficiency improvements can be made.

Instead, can possibly use pretrained model on data. Split into training/test and use model to make predictions. Currently using pixel values as feature data for model but can use actual images instead, either in the wild or cropped.

https://www.kaggle.com/datasets/abhikjha/utk-face-cropped

https://susanqq.github.io/UTKFace/

possible race trained model?
https://colab.research.google.com/drive/1MMTe7XPvvVuweAxBcc_erHNV1haWoH4S?usp=sharing

In [1]:
#!pip install deepface

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from PIL import Image
#from tensorflow.keras.applications import ResNet50
#from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
#from tensorflow.keras.preprocessing import image
# Face recognition and facial attribute analysis framework
#from deepface import DeepFace

2024-05-25 06:16:36.081457: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-25 06:16:36.132602: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Load pre-trained ResNet50 model trained on ImageNet
#model = ResNet50(weights='imagenet')

In [4]:
# Dataframe containing a list of 5 attributes for each image.
# In particular, race is labelled with a number from 0-4, 
# denoting White, Black, Asian, Indian, and Others (like Hispanic, Latino, Middle Eastern)
face_attr = pd.read_csv('age_gender.csv')

In [5]:
face_attr

,age,ethnicity,gender,img_name,pixels
0,1,2,0,20161219203650636.jpg.chip.jpg,129 128 128 126 127 130 133 135 139 142 145 14...
1,1,2,0,20161219222752047.jpg.chip.jpg,164 74 111 168 169 171 175 182 184 188 193 199...
2,1,2,0,20161219222832191.jpg.chip.jpg,67 70 71 70 69 67 70 79 90 103 116 132 145 155...
3,1,2,0,20161220144911423.jpg.chip.jpg,193 197 198 200 199 200 202 203 204 205 208 21...
4,1,2,0,20161220144914327.jpg.chip.jpg,202 205 209 210 209 209 210 211 212 214 218 21...
...,...,...,...,...,...
23700,99,0,1,20170120221920654.jpg.chip.jpg,127 100 94 81 77 77 74 99 102 98 128 145 160 1...
23701,99,1,1,20170120134639935.jpg.chip.jpg,23 28 32 35 42 47 68 85 98 103 113 117 130 129...
23702,99,2,1,20170110182418864.jpg.chip.jpg,59 50 37 40 34 19 30 101 156 170 177 184 187 1...
23703,99,2,1,20170117195405372.jpg.chip.jpg,45 108 120 156 206 197 140 180 191 199 204 207...


In [6]:
# 48 x 48 pixel images
len(face_attr['pixels'][0].split(' '))

2304

In [7]:
face_attr['ethnicity'].value_counts()

ethnicity
0    10078
1     4526
3     3975
2     3434
4     1692
Name: count, dtype: int64

In [8]:
files = os.listdir('utkcropped')
directory = 'utkcropped'
images = []
labels = []
for filename in files:
    new_filename = filename.split('_')[-1]
    img_path = os.path.join(directory, filename)
    matching_row = face_attr[face_attr['img_name'] == new_filename]
    
    try:
        if not matching_row.empty:
            ethnicity = matching_row.iloc[0]['ethnicity']
            img = Image.open(img_path)  # Open the image
            img_array = np.array(img)  # Convert the image to a NumPy array
            # Append the new name, image array, and additional data to the list
            images.append(img_array)
            labels.append(ethnicity)
        else:
            print(f"No matching data found for {new_filename}")
    except Exception as e:
        continue

No matching data found for 20170116174525125.jpg.chip.jpg
No matching data found for 20170109150557335.jpg.chip.jpg
No matching data found for utkcropped
No matching data found for 20170109150557335.jpg.chip.jpg
No matching data found for 20170109142408075.jpg.chip.jpg


In [12]:
X = np.array(images)
y = np.array(labels)
y = to_categorical(y, len(np.unique(face_attr['ethnicity'])))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
num_classes = len(y[0])

# Example CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200,200,3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate on test set if available
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

2024-05-25 06:19:06.521341: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9622 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:61:00.0, compute capability: 7.5


Epoch 1/10


2024-05-25 06:19:11.432204: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-05-25 06:19:12.261952: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f67e13d8990 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-25 06:19:12.262069: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-05-25 06:19:12.279788: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-25 06:19:12.456461: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


475/475 [==============================] - 16s 24ms/step - loss: 5.5242 - accuracy: 0.5142 - val_loss: 1.0125 - val_accuracy: 0.6325
Epoch 2/10
475/475 [==============================] - 11s 23ms/step - loss: 0.9626 - accuracy: 0.6539 - val_loss: 0.9088 - val_accuracy: 0.6805
Epoch 3/10
475/475 [==============================] - 10s 21ms/step - loss: 1.0251 - accuracy: 0.6406 - val_loss: 0.9855 - val_accuracy: 0.6517
Epoch 4/10
475/475 [==============================] - 10s 22ms/step - loss: 0.8140 - accuracy: 0.7127 - val_loss: 0.8616 - val_accuracy: 0.6994
Epoch 5/10
475/475 [==============================] - 11s 22ms/step - loss: 0.7133 - accuracy: 0.7427 - val_loss: 0.9814 - val_accuracy: 0.6852
Epoch 6/10
475/475 [==============================] - 11s 22ms/step - loss: 0.6292 - accuracy: 0.7789 - val_loss: 0.9216 - val_accuracy: 0.6989
Epoch 7/10
475/475 [==============================] - 11s 22ms/step - loss: 0.5282 - accuracy: 0.8123 - val_loss: 1.0712 - val_accuracy: 0.6976
Epo

In [14]:
# Make predictions
predictions = model.predict(X_test)

# Convert predictions from one-hot encoded back to categorical labels
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=1)

149/149 [==============================] - 1s 6ms/step


In [19]:
unique_elements, counts = np.unique(predicted_labels, return_counts=True)

# Print the predicted label counts
for element, count in zip(unique_elements, counts):
    print(f"{element}: {count}")

0: 2340
1: 911
2: 662
3: 707
4: 121


In [20]:
unique_elements, counts = np.unique(true_labels, return_counts=True)

# Print the true label counts
for element, count in zip(unique_elements, counts):
    print(f"{element}: {count}")

0: 1963
1: 917
2: 717
3: 819
4: 325


Race is labelled with a number from 0-4, denoting White, Black, Asian, Indian, and Others (like Hispanic, Latino, Middle Eastern)

Here, the model is doing a fairly decent job predicting the race/ethnicity labels in the data with ~68% accuracy. However, if we look at the predicted labels and compare them to the true labels, we can see that the model is actually overpredicting one specific race (white) by a significant margin (almost 20% higher than it occurs in reality).

This is sort of what I think would be good to demonstrate in the demo? Essentially, if we train our model on some disproportionate/biased data (original dataset is primarily composed of white people), we can get biased model outputs.